### Colabos notebook, ahol tudtam finetunolni modellt

In [ ]:
%pip install openai --quiet
%pip install langchain --quiet
%pip install chromadb --quiet
%pip install tiktoken --quiet
%pip install transformers --quiet

### Paraméterek

In [ ]:
# használt erőforrások mappája
res_folder = 'res/in_use/'

# splitting paraméterek
chunk_size = 500
chunk_overlap = 50

# vector store paraméterek
persist_directory = "res/chroma/"
search_type = "mmr"
search_k = 5
search_fetch_k = 10
lambda_mult = 0.8

# memória
memory_k = 3

# model paraméterek
temperature = 0.4
max_tokens = 1000
# githubra nem lehet feltölteni, mert túl nagy
model_name = "../mt5 training/full_trained_default"

## Erőforrásfájlok betöltése

Ha már egyszer megtettük és van mentett vektor adatbázis, akkor nem kell újra futtatni.
### CSV fájlok (nagyrészt kérdések) betöltése

In [ ]:
from langchain.document_loaders import CSVLoader, DirectoryLoader

directory_loader = DirectoryLoader(res_folder, glob="*.csv", use_multithreading=True, loader_cls=CSVLoader, loader_kwargs={"encoding": "utf-8"})
csv_data = directory_loader.load()
print(len(csv_data))
#csv_data


### Szöveges fájlok betöltése, majd feldarabolása

In [ ]:
from langchain.document_loaders import DirectoryLoader, TextLoader

directory_loader = DirectoryLoader(res_folder, glob="*.txt", use_multithreading=True, loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
text_data = directory_loader.load()
#text_data

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=True,
    separators=["\n\s*\n", "\n\s*", "\n"]
)

split_text_data = text_splitter.split_documents(text_data)
print(len(split_text_data))
#split_text_data

### VectorStore inicializálása Chroma-val

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma

oai_embedding = OpenAIEmbeddings()

In [ ]:
# Elég egyszer futtatni, ha nem változtatunk az adatokon, mert lementi a vektoradatbázist.
combined_data = []
combined_data.extend(split_text_data)
combined_data.extend(csv_data)

In [ ]:
oai_vectordb = Chroma.from_documents(
    documents=combined_data,
    embedding=oai_embedding,
    persist_directory=persist_directory
)

oai_vectordb.persist()
vectordb = oai_vectordb

Ha le van már mentve, lehet ezt használni az előző cella  helyett:

In [ ]:
oai_vectordb = Chroma(persist_directory=persist_directory, embedding_function=oai_embedding)
vectordb = oai_vectordb

In [ ]:
# MMR teszt
question = "Miket kell leadni a gyakorlathoz?"
search_result = vectordb.max_marginal_relevance_search(question,k = search_k, fetch_k = search_fetch_k, lambda_mult = lambda_mult)
print(search_result)
print(vectordb.similarity_search_with_relevance_scores(question))

### Memória config a chat historyhoz

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k = memory_k, memory_key="chat_history", return_messages=True)

## Chatbot

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, temperature=0.4, max_length=50, top_k=20, top_p=1)
tokenizer = AutoTokenizer.from_pretrained('google/mt5-base')
pipe = pipeline(task="text2text-generation", model=model, tokenizer=tokenizer)

### Alap ConversationalRetrievalChain használatával

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)
memory.clear()

chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever = vectordb.as_retriever(
        searh_type = search_type,
        search_kwargs = {
             "k": search_k,
             "fetch_k": search_fetch_k
             }
    ),
    verbose = True,
    memory = memory
)

### Saját prompttal

Az előző cella helyett lehet ezt használni, ha nem a beépített dolgot szeretnénk használni (jobban is működik magyarhoz):

In [ ]:
from langchain.chains import StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

llm = HuggingFacePipeline(pipeline=pipe)

template = """A chat előzményekből és egy következő kérdésből álló input alapján alakítsd át a következő kérdést akkor, ha a kérdés teljes értelmezéséhez szükséges a korábbi kontextus is, úgy, hogy az új kérdés értelmezhető legyen magában is.
Nem fogalmmazz új kérdést bele a korábbi kontextus alapján, csak alakítsd át a kérdést, ha szükséges. Ha nem kapcsolódik szakmai gyakorlathoz a beszélgetés, akkor írd be, hogy "Erre sajnos nem tudsz válaszolni".
Chat előzmények:
    {chat_history}

Eredeti kérdés: {question}
Új kérdés:"""
question_generator_prompt = PromptTemplate.from_template(template)
question_generator_chain = LLMChain(
    llm=llm,
    prompt=question_generator_prompt,
    #verbose=True
)


In [ ]:
from langchain.prompts import PromptTemplate

qa_template = """A BME VIK szakmai gyakorlat kérdéseire válaszoló chatbot vagy. A kérdésekre magyarul válaszolj!
Használd az alábbi dokumentumrészleteket forrásként és a chat előzményeket a kérdés megválaszolásához! Ha azokból nem tudsz megadni releváns választ, akkor válaszold azt, hogy "Sajnos erre nem tudok válaszolni, kérdezz mást a BME VIK szakmai gyakorlattal kapcsolatban!".

Dokumentumok:
    {context}

Chat előzmények:
    {chat_history}

Kérdés: {question}
Válasz:"""

qa_prompt = PromptTemplate.from_template(qa_template, input_variable_name=["question", "chat_history"])
llm_chain = LLMChain(
    llm=llm,
    prompt=qa_prompt,
    verbose=True
)
combine_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    verbose=True
)


In [ ]:
memory.clear()
chain = ConversationalRetrievalChain(
    combine_docs_chain=combine_docs_chain,
    retriever = vectordb.as_retriever(
        searh_type = search_type,
        search_kwargs = {
             "k": search_k,
             "fetch_k": search_fetch_k
             }
    ),
    question_generator=question_generator_chain,
    memory=memory
)

## Gradio UI a chatbothoz

In [ ]:
%pip install gradio --quiet

In [ ]:
message="Ki a szakmai gyakorlat felelőse?"
pipe(message)[0]['generated_text']

In [ ]:
def qa(message, history) -> str:
    return pipe(message)[0]['generated_text']

In [ ]:
import gradio as gr

chat_ui = gr.ChatInterface(qa, title = "Lacibot", description="Kérdezz a VIK-es szakmai gyakorlatról!", undo_btn=None)
chat_ui.launch()

## Tesztelés


In [ ]:
memory.buffer

In [ ]:
memory.clear()

In [ ]:
question = "Ki a tárgyfelelős?"
chain({"question": question, "chat_history": memory.buffer})


#### Teszt kérdések betöltése

In [ ]:
%pip install pandas --quiet
%pip install matplotlib --quiet

In [ ]:
import pandas as pd

test_questions = pd.read_csv("testing/test_questions.csv")
print(len(test_questions))
#test_questions

### Full tesztek

In [ ]:
import pandas as pd
from langchain.evaluation.qa import QAEvalChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
eval_chain = QAEvalChain.from_llm(llm)


Kérdések egyenként

In [ ]:
df = pd.DataFrame(columns=['Question', 'Human Answer', 'AI Answer', 'AI Evaluation'])
for i, question in enumerate(test_questions['Question']):
    memory.clear() # reset memory, hogy ne legyen hatása a következő kérdésre
    ai_ans = chain({"question": question, "chat_history": memory.buffer})
    eval_result = eval_chain({"query": question, "result": ai_ans['answer'], "answer": test_questions['Answer'][i]})
    df.loc[len(df)] = [question, test_questions['Answer'][i], ai_ans['answer'], eval_result['results']]

df

In [ ]:
# elmentés
df.to_csv(f"testing/models/mGPT_k{search_k}_size{chunk_size}.csv", index=False)

Összefüggő beszélgetés

In [ ]:
test_chat = pd.read_csv("testing/test_chat.csv")
print(len(test_chat))
#test_chat

In [ ]:
memory.clear()
chain.verbose=True
df = pd.DataFrame(columns=['Question', 'Human Answer', 'AI Answer', 'AI Evaluation'])
for i, question in enumerate(test_chat['Question']):
    ai_ans = chain({"question": question, "chat_history": memory.buffer})
    eval_result = eval_chain({"query": question, "result": ai_ans['answer'], "answer": test_chat['Answer'][i]})
    df.loc[len(df)] = [question, test_chat['Answer'][i], ai_ans['answer'], eval_result['results']]

df

In [ ]:
# elmentés
df.to_csv(f"testing/models/mGPT_CHAT_k{search_k}_size{chunk_size}.csv", index=False)